In [14]:
pip install pytesseract pillow opencv-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pytesseract
import cv2
from PIL import Image

# OCR을 수행할 이미지 파일명
image_filename = "4.PNG"
image_path = os.path.abspath(image_filename)

# 이미지 열기 (OpenCV 사용)
try:
    img = cv2.imread(image_path)

    # 이미지 전처리: 흑백 변환 (Grayscale)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 이미지 전처리: 이진화 (Thresholding)
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 배경 제거 (Adaptive Thresholding)
    adaptive_thresh = cv2.adaptiveThreshold(
        binary, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )

    # ✅ 노이즈 제거 (Gaussian Blur)
    denoised = cv2.GaussianBlur(adaptive_thresh, (3, 3), 0)

    # ✅ OCR 실행 (한국어 + 영어 지원) & 무작위 문자 방지 (--psm 6 설정)
    config = "--psm 6"
    extracted_text = pytesseract.image_to_string(denoised, lang="eng+kor", config=config)

    # 결과 출력
    print(f"📄 {image_filename} - 추출된 텍스트:")
    print(extracted_text)

except FileNotFoundError:
    print(f"❌ 오류: '{image_filename}' 파일을 찾을 수 없습니다.")

except Exception as e:
    print(f"❌ 오류: {e}")


📄 4.PNG - 추출된 텍스트:
rt
ote) ㅇ
~
3rdwagle-5 =
= 06
@
@ 지 영인 저 장 소 의 Branch Protection Rules= 인해 푸 시 가 차 단 되었습니다.
제 계 정 을 푸시 가 능 하 도록 추 가 해 주실 수 있나요? 0208
O=
692 [프로젝트 흐름 정리]
1. ZEE (Android)
- 사 진 에서 일정 정보 (날짜, 시간, 이 벤 트 명 등) 추출
- 추출한 정보 백 엔 드로 전송
2. 백 엔 드 (4006.15 + GPT API)
- 받은 일정 정 보 를 (6『7 에 전 달 하여 일정한 ISON 형 태 로 변환
- 변 환 된 ISON 데 이 터 를 프 론 트 에 반환
3. ZEE (Android)
- 백 엔 드 에서 받은 JSON 일정 BSS Google Calendar APIO 등록
채 진 성 에게 답장
@ 지 영인 저 장 소 의 Branch Protection Rules= 인해 푸 시 가 차 단 되었습니다.
제 계 정 을 푸시 가 능 하 도록 추 가 해 주실 수 있나요?
늦 은 답변 죄 송 합니다! 030( 레 포 지 토 리 에서 동 일 하게 문제 발 생 하 는 것 일까요? ~)
|
©Pv1 전송

